# Attempt to remove charging artifacts

Uses different line-by-line techniques in an attempt to remove charging artifacts.

First method is FFT filter

Second method is by fitting and subtracting a polynomial to the difference between a scan and previous scans

Both methods work well and give similar results. They appear to suppress charging artifacts but they introduce new horizontal features.



Load single images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import ipywidgets

In [ ]:
data =  tifffile.imread('2022-02-22_20_12_44_1_18_mode_0.tif')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data, cmap='gray')

In [ ]:
data.shape

In [ ]:
data.dtype

## Flatten: line-by line corrections

line fit

In [ ]:
x= np.arange(data.shape[1])

In [ ]:
#Try with one line
data_line =  data[1400,:]

In [ ]:
plt.plot(data_line)

In [ ]:
linefit_params = np.polyfit(x,data_line,1)
linefit_params

In [ ]:
#Subtract line
line_y = linefit_params[1]+x*linefit_params[0]
data_lineflatten = data_line-line_y

In [ ]:
plt.plot(data_line)
plt.plot(data_lineflatten)

in a function

In [ ]:
def flatten_line(data_line):
    x= np.arange(data_line.shape[0])
    linefit_params = np.polyfit(x,data_line,1)
    
    #Subtract line
    line_y = linefit_params[1]+x*linefit_params[0]
    data_lineflatten = data_line-line_y

    return data_lineflatten

In [ ]:
#Test it
data_lineflatten = flatten_line(data_line)
plt.plot(data_line)
plt.plot(data_lineflatten)

Parabola fit

Similar to line fit but with quadratic polynomial fit

In [ ]:
def flatten_parabola(data_line):
    x= np.arange(data_line.shape[0])
    poly_params = np.polyfit(x,data_line,2)
    
    #Subtract line
    line_y = poly_params[2]+x*poly_params[1] + x*x*poly_params[0]
    data_flatten = data_line-line_y

    return data_flatten

In [ ]:
#Test it
data_lineflatten_parab = flatten_parabola(data_line)
plt.plot(data_line)
plt.plot(data_lineflatten_parab)

Try fit a spline

In [ ]:
from scipy.interpolate import UnivariateSpline

In [ ]:
spl = UnivariateSpline(x, data_line)
spl

In [ ]:
spline = spl(x)

In [ ]:
data_line

In [ ]:
spline

In [ ]:
plt.plot(data_line)
plt.plot(spline)

not a good way to remove charging

Try FFT filtering

In [ ]:
data_line_fft = np.fft.rfft(data_line)

In [ ]:
data_line_fft_powersp = np.absolute(data_line_fft)

In [ ]:
plt.plot(data_line_fft_powersp)

Check inverse fft restores signal

In [ ]:
data_line_fft_ifft = np.fft.irfft(data_line_fft)

In [ ]:
plt.plot(data_line_fft_ifft) #Restores signal ok

Try filter low frequencies

In [ ]:
filterfreq = 5
data_line_fft_filter= np.copy(data_line_fft)
data_line_fft_filter[:filterfreq] = 0 #high pass filter

data_filter = np.fft.irfft(data_line_fft_filter)

In [ ]:
plt.plot(data_filter)
plt.plot(data_line)


Looks quite good, and filter can maybe be adjusted

## FFT 1D high pass filter with adjustable parameter

In [ ]:
import ipywidgets

In [ ]:
def HighPassFilterAndPlot(freq=5):
    data_line_fft_filter= np.copy(data_line_fft)
    data_line_fft_filter[:freq] = 0 #high pass filter

    data_filter = np.fft.irfft(data_line_fft_filter)

    plt.plot(data_filter)
    plt.plot(data_line)

In [ ]:
ipywidgets.interact(HighPassFilterAndPlot, freq= (0, 100, 1) )

## Create similar filter but for the whole image

Moved to file 'AnalyseImageStack.ipynb'

In [ ]:
data_1Dfft = np.fft.rfft(data, axis=1)

In [ ]:
def getFilterData2(freq=5):
    data_fft_filter= np.copy(data_1Dfft)
    data_fft_filter[:,:freq] = 0 #high pass filter
    data_filter2 = np.fft.irfft(data_fft_filter, axis=1)
    return data_filter2


In [ ]:
fig, (ax0,ax1) = plt.subplots(1,2,sharey=True, figsize=(10,10))
ax0.imshow(data)
ax1.imshow(getFilterData2())

In [ ]:
data_filter_1dfft = getFilterData2()
plt.figure(figsize=(10,10))
plt.imshow(data_filter_1dfft ,cmap='gray')

In [ ]:
def HighPassFilterAndPlot1(freq=5):
    # fig0, (ax0,ax1) = plt.subplots(2,1,sharey=True, figsize=(20,10))
    # ax1.imshow(getFilterData2(freq))
    plt.figure(figsize=(10,10))
    plt.imshow(getFilterData2(freq))

ipywidgets.interact(HighPassFilterAndPlot1, freq= (0, 50, 1),continuous_update=False )

It appears to be working well with freq=5

# Filter scan lines using previous lines

Try to filter each individual scan line by doing subtracting a polynomial using information from previous lines.
Scan lines are very similar to each others except when there is charging occuring.
This technique works in the following way:
1. use current scan line and subtract with previous scan line(s) to get a (noisy) difference-scan.
2. Fit a polyonomial
3. Subtract this polynomial to the current line and store it
4. Next line

In [ ]:
data_lineprev_filter = np.copy(data).astype(np.float32)
nlines = data_lineprev_filter.shape[0]
x= np.arange(data_lineprev_filter.shape[1])

polyorder = 10

for i in range(1,nlines):
    curline = data_lineprev_filter[i,:]
    prevline = data_lineprev_filter[i-1,:]

    diffline = curline - prevline

    #Fit polynomial
    fitdiffline = np.poly1d(np.polyfit(x,diffline,polyorder))(x)

    newline = curline - fitdiffline

    #store the new line
    data_lineprev_filter[i,:] = newline

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data_lineprev_filter, cmap='gray')
plt.title(f'filter using poly order {polyorder} and previous line as reference')

In [ ]:
def filter_scandata2_lineprev(data0, polyorder=4):
    data_lineprev_filter = np.copy(data0).astype(np.float32)
    nlines = data_lineprev_filter.shape[0]
    x= np.arange(data_lineprev_filter.shape[1])

    for i in range(1,nlines):
        curline = data_lineprev_filter[i,:]
        prevline = data_lineprev_filter[i-1,:]

        diffline = curline - prevline

        #Fit polynomial
        fitdiffline = np.poly1d(np.polyfit(x,diffline,polyorder))(x)

        newline = curline - fitdiffline

        #store the new line
        data_lineprev_filter[i,:] = newline

    return data_lineprev_filter

In [ ]:
def filter_scandata2_lineprev_plot(polyorder):
    fig,ax = plt.subplots(1,1,figsize=(10,10))
    ax.imshow(filter_scandata2_lineprev(data,polyorder))
ipywidgets.interact(filter_scandata2_lineprev_plot, polyorder= (1, 10,1),continuous_update=False )

Try average previous lines

In [ ]:
data_lineprev_filter_nlines = np.copy(data).astype(np.float32)
nlines = data_lineprev_filter_nlines.shape[0]
x= np.arange(data_lineprev_filter_nlines.shape[1])

polyorder = 10

nlinesaverage = 10

for i in range(nlinesaverage,nlines):
    curline = data_lineprev_filter_nlines[i,:]
    prevline = np.mean(data_lineprev_filter_nlines[i-nlinesaverage:i,:], axis=0) #Average aling the y-axis

    diffline = curline - prevline

    #Fit polynomial
    fitdiffline = np.poly1d(np.polyfit(x,diffline,polyorder))(x)

    newline = curline - fitdiffline

    #store the new line
    data_lineprev_filter_nlines[i,:] = newline

In [ ]:
plt.figure(figsize= (10,10))
plt.imshow(data_lineprev_filter_nlines, cmap='gray')
plt.title(f'filter using poly order {polyorder} and previous {nlinesaverage} lines averaged as reference')

In [ ]:
data_lineprev_filter.dtype

Try doing outlier removal

In [ ]:
#Test on a single line
i = 2580
nlinesaverage= 20

curline = data[i,:].astype(np.float32)
prevline = np.mean(data[i-nlinesaverage:i,:].astype(np.float32),axis=0)


In [ ]:
plt.plot(curline)
plt.plot(prevline)

In [ ]:
diffline = curline - prevline

In [ ]:
plt.plot(diffline)

In [ ]:
#Fit polynomial
fitpolyline = np.poly1d(np.polyfit(x, diffline,polyorder))(x)
newline = curline - fitpolyline

In [ ]:
plt.plot(curline)
plt.plot(newline)

In [ ]:
plt.plot(fitpolyline)

In [ ]:
newlinecheck = newline-prevline
varline = newlinecheck**2

In [ ]:
plt.plot(varline)

In [ ]:
maskline = varline<500

In [ ]:
x_mask = x[maskline]

In [ ]:
diffline_mask = diffline[maskline]

In [ ]:
#Do the polyline fit now
fitpolyline1 = np.poly1d(np.polyfit(x_mask, diffline_mask,polyorder))(x)

In [ ]:
plt.plot(fitpolyline1)

In [ ]:
polyfitsdiff = fitpolyline1 - fitpolyline
plt.plot(polyfitsdiff)

In [ ]:
newline1 = curline - fitpolyline1

In [ ]:
plt.plot(curline)
plt.plot(newline1)

THIS METHOD IS NOT WORKING

In [ ]:
data_lineprev_filter_nlines_outlierrem = np.copy(data).astype(np.float32)
nlines = data_lineprev_filter_nlines_outlierrem.shape[0]
x= np.arange(data_lineprev_filter_nlines_outlierrem.shape[1])

polyorder = 10

nlinesaverage = 10
maskvar = 50

for i in range(nlinesaverage,nlines):
    curline = data_lineprev_filter_nlines_outlierrem[i,:]
    prevline = np.mean(data_lineprev_filter_nlines_outlierrem[i-nlinesaverage:i,:], axis=0) #Average aling the y-axis

    diffline = curline - prevline

    #Fit polynomial
    fitpolyline = np.poly1d(np.polyfit(x, diffline,polyorder))(x)
    newline = curline - fitpolyline

    #Check it against prevline
    newlinecheck = newline - prevline
    varline = newlinecheck**2

    maskline = varline<maskvar

    x_mask = x[maskline]
    diffline_mask = diffline[maskline]

    #Fit again, this time excluding bad regions
    fitpolyline1 = np.poly1d(np.polyfit(x_mask, diffline_mask,polyorder))(x)
    newline1 = curline - fitpolyline1

    #store the new line
    data_lineprev_filter_nlines_outlierrem[i,:] = newline1

plt.figure(figsize= (10,10))
plt.imshow(data_lineprev_filter_nlines_outlierrem, cmap='gray')
plt.title(f'filter using poly order {polyorder} and previous {nlinesaverage} lines averaged as reference, with outlier removal {maskvar}')

## Check resulting images FFT 2D

In [ ]:
data_fft2d = np.fft.fftshift(np.fft.fft2(data.astype(np.float32)))
fftimage = np.log(np.absolute(data_fft2d))


In [ ]:
fftimage.max()

In [ ]:
fftimage.min()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(fftimage, vmin =9, vmax=15)

In [ ]:
import napari
nv = napari.view_image(fftimage)

In [ ]:
data_filter_1dfft_fft2d = np.fft.fftshift(np.fft.fft2(data_filter_1dfft.astype(np.float32)))
data_filter_1dfft_fft2d_image = np.log(np.absolute(data_filter_1dfft_fft2d))
nv.add_image(data_filter_1dfft_fft2d_image)

In [ ]:
data_lineprev_filter_2dfft = np.fft.fftshift(np.fft.fft2(data_lineprev_filter.astype(np.float32)))
data_lineprev_filter_2dfft_image = np.log(np.absolute(data_lineprev_filter_2dfft))
nv.add_image(data_lineprev_filter_2dfft_image)

In [ ]:
data_lineprev_filter_nlines_2dfft = np.fft.fftshift(np.fft.fft2(data_lineprev_filter_nlines.astype(np.float32)))
data_lineprev_filter_nlines_2dfft_image = np.log(np.absolute(data_lineprev_filter_nlines_2dfft))
nv.add_image(data_lineprev_filter_nlines_2dfft_image)